___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Авторские права принадлежат Pierian Data Inc.</em></center>
<center><em>Для дополнительной информации посетите наш сайт <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Проверочный проект по линейной регрессии

Мы изучили построение признаков, кросс-валидацию и поиск по сетке. Пришло время проверить Ваши новые навыки. Добро пожаловать на проверочный проект по машинному обучению! В этом проекте у нас будут достаточно подробные пошаговые инструкции. Будущие проекты будут уже в более свободной форме. Мы начнём проект с финальной версии набора данных Ames Housing, с которым мы работали в разделе "построение признаков" этого курса. Цель этого проекта - создать модель линейной регрессии, обучить её на данных с поиском оптимальных значений параметров по сетке, и затем оценить модель на тестовом наборе данных.

---
---
---
## Выполните задания, написанные жирным шрифтом

**ЗАДАНИЕ: Выполните ячейки ниже для импорта библиотек и загрузки данных. Возможно в будущем Вам понадобятся дополнительные команды import от scikit-learn.**

### Imports

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Данные

In [2]:
df = pd.read_csv(r"C:\Users\mixai\OneDrive\Рабочий стол\ITMO Магистратура\ML прочее\UNZIP_ME_FOR_NOTEBOOKS_ML_RUS_V7\DATA\AMES_Final_DF.csv")

In [3]:
df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 274 entries, Lot Frontage to Sale Condition_Partial
dtypes: float64(11), int64(263)
memory usage: 6.1 MB


**ЗАДАНИЕ: Мы будем пытаться спрогнозировать значение колонки SalePrice. Разделите данные на две части - признаки X и целевая переменная y.**

In [85]:
X, y = df.drop("SalePrice", axis=1), df["SalePrice"]

**ЗАДАНИЕ: С помощью scikit-learn разделите и X, и y на обучающий и тестовый наборы данных. Поскольку далее мы будем использовать поиск по сетке, то выделите под тестовые данные 10% от всех данных. Чтобы получить такое же разбиение данных, как и в нашем блокноте, можете использовать random_state = 101.**

In [13]:
from sklearn.model_selection import train_test_split

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)


**ЗАДАНИЕ: Признаки в этом наборе данных имеют различный масштаб и различные единицы измерения. Для оптимальной работы регрессии, выполните масштабирование признаков X. Обратите внимание на то, какие данные подавать на вход для .fit(), а какие данные на вход для .transform().**

In [87]:
from sklearn.preprocessing import StandardScaler

In [88]:
scaler = StandardScaler()

In [89]:
scaler.fit(X_train)

StandardScaler()

In [90]:
X_train_scaled = scaler.transform(X_train)

In [91]:
X_test_scaled = scaler.transform(X_test)

In [92]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)

In [93]:
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

**ЗАДАНИЕ: Мы будем использовать модель "Elastic Net". Создайте экземпляр модели ElasticNet в scikit-learn, используя параметры по умолчанию.**

In [94]:
from sklearn.linear_model import ElasticNet

In [95]:
model = ElasticNet()

**ЗАДАНИЕ: Модель Elastic Net работает с двумя основными параметрами - alpha и L1_ratio. Создайте словарь с набором различных значений этих параметров, для поиска по сетке. Вы можете выбрать различные значения параметров, но имейте ввиду, что Ваши результаты могут не совпасть с результатами в блокноте с решениями.**

In [96]:
param = {'alpha':[0.1, 0.5, 1, 2, 10, 100], "l1_ratio":[0.1, 0.5, 1, 2, 5, 10, 100]}

**ЗАДАНИЕ: С помощью scikit-learn создайте объект GridSearchCV и запустите поиск по сетке для нахождения наилучших параметров модели, используя обучающие данные (предварительно смасштабированные). [Для некоторых комбинаций параметров Вы можете получить предупреждения (warnings).](https://stackoverflow.com/questions/20681864/lasso-on-sklearn-does-not-converge)**

In [97]:
from sklearn.model_selection import GridSearchCV

In [98]:
grid = GridSearchCV(estimator=model, param_grid=param, cv = 10, scoring='neg_mean_squared_error', verbose=1)

In [99]:
grid.fit(scaled_X_train, y_train)

Fitting 10 folds for each of 42 candidates, totalling 420 fits


C:\Users\mixai\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.206e+11, tolerance: 1.537e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\mixai\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.171e+11, tolerance: 1.534e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\mixai\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.380e+11, toleranc

C:\Users\mixai\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.855e+11, tolerance: 1.567e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\mixai\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.717e+11, tolerance: 1.564e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\mixai\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.913e+11, toleranc

C:\Users\mixai\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.164e+10, tolerance: 1.546e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\mixai\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
240 fits failed out of a total of 420.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mixai\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    es

GridSearchCV(cv=10, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 0.5, 1, 2, 10, 100],
                         'l1_ratio': [0.1, 0.5, 1, 2, 5, 10, 100]},
             scoring='neg_mean_squared_error', verbose=1)

**ЗАДАНИЕ: Отобразите наилучшую комбинацию параметров для Вашей модели.**

In [101]:
grid.best_params_

{'alpha': 100, 'l1_ratio': 1}

{'alpha': 100, 'l1_ratio': 1}

**ЗАДАНИЕ: Оцените работу модели на тестовом наборе данных (предварительно смасштабированном) в 10%, которые модель ещё не видела. В блокноте с решениями мы получили MAE = $\$$14149 и RMSE = $\$$20532**

In [102]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [111]:
y_predict = grid.predict(X_test_scaled)

C:\Users\mixai\anaconda3\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but ElasticNet was fitted without feature names
  warnings.warn(


In [113]:
mean_absolute_error(y_test, y_predict)

14195.354900562168

14149.055026374837

In [116]:
np.sqrt(mean_squared_error(y_test, y_predict))

20558.508566893157

20532.890234901013

180815.53743589742

## Отличная работа!

----